<a href="https://pangeos.eu/" target="_blank">
<center><img src="../images/1-logos-pan-eu-cost.png" alt="logos" width="100%"/></center>
</a>

PANGEOS summer taining school 2024

EXAMPLE OF UNCERTAINTY PROPAGATION FOR 

# CASE 2: Retrieval of traits from spectral data

**Authors:**
Egor Prikaziuk (e.prikaziuk@utwente.nl)

# Learning objectives

**After follwing this notebook you will be able to ...**
- propagate uncertainty in reflectance to uncertainty in a vegetation (spectral) index


# TOC

1. [Load reflectance](#1)
2. [Compute NDVI](#2)
3. [Propagate uncertainty](#3)
4. [Explain](#4) 

In [2]:
import pandas as pd
import punpy

from pathlib import Path

# 1
## Load reflectance +/- U
[back to TOC](#TOC)

This example uses hyperspectral reflectance recorded with a handheld point spectrometer system Piccolo doppio. 

The propagation of uncertainty in radiance and irradiance to the uncertainty in reflectance due to 
- random effects
- systematic effects of calibration

were discussed in CASE 1 notebooks

In [3]:
df = pd.read_csv('../data/R1_ut.csv')
df.head()

,wl,refl,refl_u
0,401.82374,0.026258,0.001128
1,402.61047,0.026386,0.001135
2,403.39714,0.026713,0.001149
3,404.18373,0.026888,0.001156
4,404.97025,0.027230,0.001171


# 2
## NDVI computation
[back to TOC](#TOC)

two versions possible
- single band
- spectral response function

This example uses bands correspodning to central wavelengths of Sentinel-2 B4 and B8; band ranges https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/bands/

In [4]:
wl_red = 665  # nm  B4
wl_nir = 842  # nm  B8

In [5]:
i_red = abs((df.wl - wl_red)).argmin()
i_nir = abs((df.wl - wl_nir)).argmin()
i_red, i_nir

(np.int64(340), np.int64(575))

In [6]:
r_red = df.refl.iloc[i_red]
r_nir = df.refl.iloc[i_nir]
r_red, r_nir

(np.float64(0.0565892236607021), np.float64(0.5616192798481215))

In [7]:
ndvi = (r_nir - r_red) / (r_nir + r_red)
ndvi

np.float64(0.8169251204423319)

# 3
## propagation of uncertainty
[back to TOC](#TOC)

Same steps as in the previous notebooks
1. Monte Carlo sampler initialization
2. model function definition
3. mean value computation
4. uncertainty definition
5. uncertinaty propagation

### 3-1 Monte Carlo sampler initialization

In [8]:
prop = punpy.MCPropagation(10000)

### 3-2 model function definition

In [9]:
def clc_ndvi(red, nir):
    ndvi = (nir - red) / (nir + red)
    return ndvi

###  3-3 mean value computation

In [10]:
ndvi = clc_ndvi(r_red, r_nir)

### 3-4 uncertainty definition

In [11]:
u_red = df.refl_u.iloc[i_red]
u_nir = df.refl_u.iloc[i_nir]
u_red, u_nir

(np.float64(0.0024542763743017), np.float64(0.0240881009007566))

### 3-5 uncertinaty propagation

we are assuming no correlation between uncertainties, therefore we use `.propagate_random()` method 

function help https://punpy.readthedocs.io/en/latest/content/generated/punpy.mc.mc_propagation.MCPropagation.propagate_random.html#punpy.mc.mc_propagation.MCPropagation.propagate_random

In [12]:
ndvi_ur = prop.propagate_random(clc_ndvi, 
                              [r_red, r_nir],
                              [u_red, u_nir])
ndvi_ur

C:\Users\PrikaziukE\AppData\Local\miniconda3\envs\pangeos\Lib\site-packages\punpy\mc\mc_propagation.py:986: UserWarning: It looks like one of your input quantities is not an array or does not have the same shape as the measurand. This is not a problem, but means you likely cannot use array operations in your measurement function. You might need to set parallel_cores to 1 or higher when creating your MCPropagation object.
  warnings.warn(


np.float64(0.010171795997444444)

# 4
## meaning
[back to TOC](#TOC)

In [13]:
print(f'NDVI = {ndvi:.2f} +/- {ndvi_ur:.2f}')

NDVI = 0.82 +/- 0.01


In [14]:
ndvi_ur_rel = ndvi_ur / ndvi * 100
print(f'relative NDVI uncertainty is {ndvi_ur_rel:.1f}%')

relative NDVI uncertainty is 1.2%
